In [1]:
!pip install streamlit transformers torch  langchain sentence-transformers faiss-cpu pyvi langchain_community accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existin

In [2]:
!pip install gradio

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/MyDrive/Data\ ChatBot

faiss_tthc		postprocessed_tthc.csv	Vistral-finetuned-final
phogpt-finetuned-final	test_phogpt.jsonl
postprocessed_link.csv	train_phogpt.jsonl


In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
import torch
import os
from pyvi.ViTokenizer import tokenize

# Hugging Face Token
# HF_TOKEN = "YOUR HUGGING FACE TOKEN"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

# Config
MODEL_NAME = "Viet-Mistral/Vistral-7B-Chat"
EMBED_MODEL = "keepitreal/vietnamese-sbert"
RERANK_MODEL = "itdainb/PhoRanker"
MAX_LENGTH = 512

# Prompt Template RAG
prompt_template = """
Bạn là trợ lý ảo dịch vụ công thông minh.
Sử dụng thông tin dưới đây để trả lời câu hỏi một cách chọn lọc và chính xác.
Không được bịa đặt hoặc suy diễn thông tin ngoài nội dung đã cho.
Nếu không tìm thấy thông tin trong context, hãy trả lời: "Xin lỗi, tôi không có thông tin về điều đó."

Context: {context}
Câu hỏi: {question}
Trả lời:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Load model & tokenizer
def load_model():
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,        # hoặc load_in_8bit=True
        bnb_4bit_compute_dtype=torch.float16,  # fp16 for faster
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quantization_config,
        device_map="auto",
        use_auth_token=HF_TOKEN
    )
    return tokenizer, model

print("load model...")
tokenizer, model = load_model()
print("Model đã load thành công!")

# Pipeline for text generation
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    repetition_penalty=1.1,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Load embeddings and FAISS database
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
db = FAISS.load_local("/content/drive/MyDrive/Data ChatBot/faiss_tthc", embeddings, allow_dangerous_deserialization=True)

# Load model rerank
model_rerank = CrossEncoder(RERANK_MODEL, max_length=MAX_LENGTH)

#  RAG and rerank
def rag_rerank(user_query, history):
    docs = db.similarity_search(user_query, k=10)
    all_texts = [d.page_content for d in docs]

    tokenized_query = tokenize(user_query)
    tokenized_sentences = [tokenize(sent) for sent in all_texts]
    tokenized_pairs = [(tokenized_query, sent) for sent in tokenized_sentences]

    scores = model_rerank.predict(tokenized_pairs)
    ranked_docs = [doc for _, doc in sorted(zip(scores, all_texts), key=lambda x: x[0], reverse=True)]

    top_docs = "\n\n".join(ranked_docs[:3])
    final_prompt = PROMPT.format(context=top_docs, question=user_query)

    answer = llm_pipeline(final_prompt)[0]['generated_text']
    history.append((user_query, answer))
    return history, ""

#UI Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 💬 Chatbot TTHC (Vistral-7B)")

    chatbot = gr.Chatbot(height=500)
    msg = gr.Textbox(placeholder="Nhập câu hỏi của bạn...", label="Câu hỏi")
    clear = gr.Button("Xóa hội thoại")

    msg.submit(rag_rerank, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: [], None, chatbot)

demo.launch(share=True)


🔄 Đang load model...


tokenizer_config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

...

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

/tmp/ipython-input-3190467174.py:99: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a56652511e840542a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
user_query = "Lệ phí cấp lại thẻ Căn cước công dân là bao nhiêu?"

docs = db.similarity_search(user_query, k=10)
all_texts = [d.page_content for d in docs]

tokenized_query = tokenize(user_query)
tokenized_sentences = [tokenize(sent) for sent in all_texts]
tokenized_pairs = [(tokenized_query, sent) for sent in tokenized_sentences]

scores = model_rerank.predict(tokenized_pairs)
ranked_docs = [doc for _, doc in sorted(zip(scores, all_texts), key=lambda x: x[0], reverse=True)]

top_docs = "\n\n".join(ranked_docs[:3])
final_prompt = PROMPT.format(context=top_docs, question=user_query)

answer = llm_pipeline(final_prompt)[0]['generated_text']


In [8]:
top_docs

'Tên thủ tục: Cấp thẻ Căn cước công dân khi chưa có thông tin trong Cơ sở dữ liệu quốc gia về dân cư (thực hiện tại cấp trung ương)\ncachThucThucHien: Hình thức nộp Thời hạn giải quyết Phí, lệ phí Mô tả\r\nTrực tiếp 07 Ngày làm việc Lệ phí : 30.000 Đồng Công dân chuyển từ Chứng minh nhân dân 9 số, Chứng minh nhân dân 12 số sang cấp thẻ Căn cước công dân: 30.000 đồng/ thẻ Căn cước công dân.\r\nLệ phí : 0 Đồng Công dân từ đủ 14 tuổi trở lên làm thủ tục cấp thẻ Căn cước công dân lần đầu không phải nộp lệ phí\r\n+ Trực tiếp tại trụ sở Trung tâm dữ liệu quốc gia về dân cư, Cục Cảnh sát quản lý hành chính về trật tự xã hội, Bộ Công an; + Thông qua Cổng dịch vụ công quốc gia, Cổng dịch vụ công Bộ Công an để đăng ký thời gian, địa điểm làm thủ tục đề nghị cấp thẻ Căn cước công dân; + Cấp lưu động tại các địa điểm cần thiết\n\nTên thủ tục: Cấp thẻ Căn cước công dân khi chưa có thông tin trong Cơ sở dữ liệu quốc gia về dân cư (thực hiện tại cấp huyện)\ncachThucThucHien: Hình thức nộp Thời hạn gi